In [19]:
%%sh
pip install pycaret
pip install python-dotenv
pip install ydata-profiling
pip install shap
pip -q install --upgrade stepfunctions
pip install s3fs

#TODO this should be in requirments.txt

DEBUG:asyncio:Using selector: KqueueSelector



[notice] A new release of pip is available: 23.3.2 -> 24.2
[notice] To update, run: python3.10 -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.2 -> 24.2
[notice] To update, run: python3.10 -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.2 -> 24.2
[notice] To update, run: python3.10 -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.2 -> 24.2
[notice] To update, run: python3.10 -m pip install --upgrade pip
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.5.4 requires botocore<1.31.18,>=1.31.17, but you have botocore 1.31.85 which is incompatible.
sagemaker-training 4.7.3 requires botocore==1.31.57, but you have botocore 1.31.85 which is incompatible.

[notice] A new release of pip is available: 23.3.2 -> 24.2
[notice] To update, run: python3.10 -m pip install --upgrade pip


  Using cached botocore-1.31.17-py3-none-any.whl.metadata (5.9 kB)
Using cached botocore-1.31.17-py3-none-any.whl (11.1 MB)
  Attempting uninstall: botocore
    Found existing installation: botocore 1.31.85
    Uninstalling botocore-1.31.85:
      Successfully uninstalled botocore-1.31.85


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
boto3 1.28.57 requires botocore<1.32.0,>=1.31.57, but you have botocore 1.31.17 which is incompatible.
sagemaker-training 4.7.3 requires botocore==1.31.57, but you have botocore 1.31.17 which is incompatible.



[notice] A new release of pip is available: 23.3.2 -> 24.2
[notice] To update, run: python3.10 -m pip install --upgrade pip


In [1]:
# TODO wrap this whole script in a retraining.py file to be scheduled on a regular basis in e.g Glue or Lambda (depending how long it takes to run it)

import boto3
import sagemaker
import uuid
import logging
import stepfunctions
import os

from sagemaker.amazon.amazon_estimator import get_image_uri
from stepfunctions import steps
from stepfunctions.inputs import ExecutionInput
from stepfunctions.workflow import Workflow

from sagemaker import get_execution_role
from dotenv import load_dotenv
from load_data import load_data
from transform_data import split_data
from time import strftime, gmtime
from io import StringIO

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/lanre.bakare/Library/Application Support/sagemaker/config.yaml


In [2]:
logging.basicConfig(level=logging.DEBUG)
timestamp = strftime("%d-%H-%M-%S", gmtime())

In [3]:
# Variables Setup Stage
load_dotenv(".env")
role = get_execution_role()

# Env variables
data_location_s3 = os.getenv("data_location_s3")
algorithm_choice = os.getenv("algorithm_choice")
target = os.getenv("target")
endpoint_name = os.getenv("endpoint_name")
model_name = os.getenv("model_name")
data_location = "s3://{}".format(data_location_s3)
instance_type = os.getenv("instance_type")
model_instance_count = int(os.getenv("model_instance_count"))
image_uri = os.getenv("ecr_repo_uri")
tuning_metric = os.getenv("tuning_metric")
ml_model = "xgboost"
ml_repo_version = "0.90-2"

algorithm_choice_dict = {"classification": "binary:logistic", "regression": "reg:squarederror"}

logging.info(
    f"{data_location_s3}, {algorithm_choice}, {target}, {endpoint_name}, {model_name}, {data_location}, {instance_type}, {image_uri}, {tuning_metric}, {role}"
)

DEBUG:botocore.hooks:Changing event name from creating-client-class.iot-data to creating-client-class.iot-data-plane
DEBUG:botocore.hooks:Changing event name from before-call.apigateway to before-call.api-gateway
DEBUG:botocore.hooks:Changing event name from request-created.machinelearning.Predict to request-created.machine-learning.Predict
DEBUG:botocore.hooks:Changing event name from before-parameter-build.autoscaling.CreateLaunchConfiguration to before-parameter-build.auto-scaling.CreateLaunchConfiguration
DEBUG:botocore.hooks:Changing event name from before-parameter-build.route53 to before-parameter-build.route-53
DEBUG:botocore.hooks:Changing event name from request-created.cloudsearchdomain.Search to request-created.cloudsearch-domain.Search
DEBUG:botocore.hooks:Changing event name from docs.*.autoscaling.CreateLaunchConfiguration.complete-section to docs.*.auto-scaling.CreateLaunchConfiguration.complete-section
DEBUG:botocore.hooks:Changing event name from before-parameter-buil

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/lanre.bakare/Library/Application Support/sagemaker/config.yaml


DEBUG:botocore.loaders:Loading JSON file: /Users/lanre.bakare/Documents/internal-projects/terraform-aws-mlops-module/venv/lib/python3.10/site-packages/botocore/data/endpoints.json
DEBUG:botocore.hooks:Event choose-service-name: calling handler <function handle_service_name_alias at 0x105344c10>
DEBUG:botocore.loaders:Loading JSON file: /Users/lanre.bakare/Documents/internal-projects/terraform-aws-mlops-module/venv/lib/python3.10/site-packages/botocore/data/sts/2011-06-15/service-2.json
DEBUG:botocore.loaders:Loading JSON file: /Users/lanre.bakare/Documents/internal-projects/terraform-aws-mlops-module/venv/lib/python3.10/site-packages/botocore/data/sts/2011-06-15/endpoint-rule-set-1.json.gz
DEBUG:botocore.hooks:Event creating-client-class.sts: calling handler <function add_generate_presigned_url at 0x105293a30>
DEBUG:botocore.endpoint:Setting sts timeout as (60, 60)
DEBUG:botocore.client:Registering retry handlers for service: sts
DEBUG:botocore.hooks:Event before-parameter-build.sts.Ge

In [4]:
stepfunctions.set_stream_logger(level=logging.INFO)
bucket = data_location_s3.split('/')[0]
prefix = "step_function"
bucket_path = 's3://{}/{}/'.format(bucket, prefix)
region = boto3.Session().region_name
session = sagemaker.Session()
FILE_TRAIN = "train.csv"
FILE_TEST = "test.csv"
FILE_VALIDATION = "validation.csv"
train_s3_file = os.path.join(prefix, FILE_TRAIN)
test_s3_file = os.path.join(prefix, FILE_TEST)
validation_s3_file = os.path.join(prefix, FILE_VALIDATION)
print(train_s3_file, test_s3_file, validation_s3_file)
output_s3 = "s3://{}/{}/{}/".format(bucket, prefix, "output")
print(train_s3_file, validation_s3_file, test_s3_file, output_s3)

DEBUG:botocore.hooks:Changing event name from creating-client-class.iot-data to creating-client-class.iot-data-plane
DEBUG:botocore.hooks:Changing event name from before-call.apigateway to before-call.api-gateway
DEBUG:botocore.hooks:Changing event name from request-created.machinelearning.Predict to request-created.machine-learning.Predict
DEBUG:botocore.hooks:Changing event name from before-parameter-build.autoscaling.CreateLaunchConfiguration to before-parameter-build.auto-scaling.CreateLaunchConfiguration
DEBUG:botocore.hooks:Changing event name from before-parameter-build.route53 to before-parameter-build.route-53
DEBUG:botocore.hooks:Changing event name from request-created.cloudsearchdomain.Search to request-created.cloudsearch-domain.Search
DEBUG:botocore.hooks:Changing event name from docs.*.autoscaling.CreateLaunchConfiguration.complete-section to docs.*.auto-scaling.CreateLaunchConfiguration.complete-section
DEBUG:botocore.hooks:Changing event name from before-parameter-buil

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/lanre.bakare/Library/Application Support/sagemaker/config.yaml
step_function/train.csv step_function/test.csv step_function/validation.csv
step_function/train.csv step_function/validation.csv step_function/test.csv s3://streaming-data-platform-ml-data/step_function/output/


In [5]:
FILE_DATA = load_data(data_location)
FILE_DATA.head()

DEBUG:asyncio:Using selector: KqueueSelector
DEBUG:s3fs:Setting up s3fs instance
DEBUG:botocore.hooks:Changing event name from creating-client-class.iot-data to creating-client-class.iot-data-plane
DEBUG:botocore.hooks:Changing event name from before-call.apigateway to before-call.api-gateway
DEBUG:botocore.hooks:Changing event name from request-created.machinelearning.Predict to request-created.machine-learning.Predict
DEBUG:botocore.hooks:Changing event name from before-parameter-build.autoscaling.CreateLaunchConfiguration to before-parameter-build.auto-scaling.CreateLaunchConfiguration
DEBUG:botocore.hooks:Changing event name from before-parameter-build.route53 to before-parameter-build.route-53
DEBUG:botocore.hooks:Changing event name from request-created.cloudsearchdomain.Search to request-created.cloudsearch-domain.Search
DEBUG:botocore.hooks:Changing event name from docs.*.autoscaling.CreateLaunchConfiguration.complete-section to docs.*.auto-scaling.CreateLaunchConfiguration.com

,age,job,education,default,balance,housing,loan,y
0,32,7,2,1,-238,1,0,0
1,34,4,2,0,-478,1,1,0
2,32,3,2,0,266,1,0,0
3,36,7,2,1,13,0,1,0
4,23,11,2,0,486,0,0,0


In [6]:
train_and_val_data, test_data = split_data(FILE_DATA, shuffle=True)

In [7]:
train_data, validation_data = split_data(train_and_val_data, shuffle=True)

In [8]:
def upload_to_s3(file): 
    csv_buffer = StringIO()
    train_data.to_csv(csv_buffer, index=False)
    s3_resource = boto3.resource("s3")
    s3_resource.Object(bucket, file).put(Body=csv_buffer.getvalue())


upload_to_s3(train_s3_file)
upload_to_s3(validation_s3_file)
upload_to_s3(test_s3_file)

DEBUG:botocore.hooks:Changing event name from creating-client-class.iot-data to creating-client-class.iot-data-plane
DEBUG:botocore.hooks:Changing event name from before-call.apigateway to before-call.api-gateway
DEBUG:botocore.hooks:Changing event name from request-created.machinelearning.Predict to request-created.machine-learning.Predict
DEBUG:botocore.hooks:Changing event name from before-parameter-build.autoscaling.CreateLaunchConfiguration to before-parameter-build.auto-scaling.CreateLaunchConfiguration
DEBUG:botocore.hooks:Changing event name from before-parameter-build.route53 to before-parameter-build.route-53
DEBUG:botocore.hooks:Changing event name from request-created.cloudsearchdomain.Search to request-created.cloudsearch-domain.Search
DEBUG:botocore.hooks:Changing event name from docs.*.autoscaling.CreateLaunchConfiguration.complete-section to docs.*.auto-scaling.CreateLaunchConfiguration.complete-section
DEBUG:botocore.hooks:Changing event name from before-parameter-buil

In [9]:
sagemaker_execution_role = role
xgb = sagemaker.estimator.Estimator(
    get_image_uri(
        region, ml_model, repo_version=ml_repo_version
    ), 
    sagemaker_execution_role,
    train_instance_count=1,
    train_instance_type=instance_type,
    output_path=output_s3,
    sagemaker_session=session,
)

xgb.set_hyperparameters(
    objective=algorithm_choice_dict[algorithm_choice],
    num_round=50,
    max_depth=5,
    eta=0.2,
    gamma=4,
    min_child_weight=6,
    subsample=0.7,
    silent=0,
)

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
INFO:sagemaker.image_uris:Defaulting to only available Python version: py3
DEBUG:botocore.loaders:Loading JSON file: /Users/lanre.bakare/Documents/internal-projects/terraform-aws-mlops-module/venv/lib/python3.10/site-packages/botocore/data/endpoints.json
DEBUG:botocore.regions:Creating a regex based endpoint for ecr, us-east-1
INFO:sagemaker.image_uris:Defaulting to only supported image scope: cpu.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
DEBUG:botocore.hooks:Event choose-service-name: calling handler <function handle_service_name_alias at 0x105344c10>
DEBUG:botocore.loaders:Loading JSON file: /Users/lanre.bakare/Documents/internal-projects/terraform-aws-mlops-module/venv/lib/python3.10/site-packages/botocore/data/sagemaker/2017-07-24/service-2.json
DEBUG:botocore.loaders:Loading JSON file: /Users/lanre.bakare/Documents/in

In [10]:
execution_input = ExecutionInput(schema={
    'JobName': str, 
    'ModelName': str,
    'EndpointName': str
})

In [11]:
training_step = steps.TrainingStep(
    'Train Step', 
    estimator=xgb,
    data={
        'train': sagemaker.inputs.TrainingInput(train_s3_file, content_type='csv'),
        'validation': sagemaker.inputs.TrainingInput(validation_s3_file, content_type='csv')
    },
    job_name=execution_input['JobName']
)

DEBUG:botocore.hooks:Changing event name from creating-client-class.iot-data to creating-client-class.iot-data-plane
DEBUG:botocore.hooks:Changing event name from before-call.apigateway to before-call.api-gateway
DEBUG:botocore.hooks:Changing event name from request-created.machinelearning.Predict to request-created.machine-learning.Predict
DEBUG:botocore.hooks:Changing event name from before-parameter-build.autoscaling.CreateLaunchConfiguration to before-parameter-build.auto-scaling.CreateLaunchConfiguration
DEBUG:botocore.hooks:Changing event name from before-parameter-build.route53 to before-parameter-build.route-53
DEBUG:botocore.hooks:Changing event name from request-created.cloudsearchdomain.Search to request-created.cloudsearch-domain.Search
DEBUG:botocore.hooks:Changing event name from docs.*.autoscaling.CreateLaunchConfiguration.complete-section to docs.*.auto-scaling.CreateLaunchConfiguration.complete-section
DEBUG:botocore.hooks:Changing event name from before-parameter-buil

In [12]:
model_step = steps.ModelStep(
    'Save model',
    model=training_step.get_expected_model(),
    model_name=execution_input['ModelName']  
)

DEBUG:botocore.hooks:Changing event name from creating-client-class.iot-data to creating-client-class.iot-data-plane
DEBUG:botocore.hooks:Changing event name from before-call.apigateway to before-call.api-gateway
DEBUG:botocore.hooks:Changing event name from request-created.machinelearning.Predict to request-created.machine-learning.Predict
DEBUG:botocore.hooks:Changing event name from before-parameter-build.autoscaling.CreateLaunchConfiguration to before-parameter-build.auto-scaling.CreateLaunchConfiguration
DEBUG:botocore.hooks:Changing event name from before-parameter-build.route53 to before-parameter-build.route-53
DEBUG:botocore.hooks:Changing event name from request-created.cloudsearchdomain.Search to request-created.cloudsearch-domain.Search
DEBUG:botocore.hooks:Changing event name from docs.*.autoscaling.CreateLaunchConfiguration.complete-section to docs.*.auto-scaling.CreateLaunchConfiguration.complete-section
DEBUG:botocore.hooks:Changing event name from before-parameter-buil

In [13]:
endpoint_config_step = steps.EndpointConfigStep(
    "Create Endpoint Config",
    endpoint_config_name=execution_input["ModelName"],
    model_name=execution_input["ModelName"],
    initial_instance_count=1,
    instance_type=instance_type 
)

DEBUG:botocore.hooks:Changing event name from creating-client-class.iot-data to creating-client-class.iot-data-plane
DEBUG:botocore.hooks:Changing event name from before-call.apigateway to before-call.api-gateway
DEBUG:botocore.hooks:Changing event name from request-created.machinelearning.Predict to request-created.machine-learning.Predict
DEBUG:botocore.hooks:Changing event name from before-parameter-build.autoscaling.CreateLaunchConfiguration to before-parameter-build.auto-scaling.CreateLaunchConfiguration
DEBUG:botocore.hooks:Changing event name from before-parameter-build.route53 to before-parameter-build.route-53
DEBUG:botocore.hooks:Changing event name from request-created.cloudsearchdomain.Search to request-created.cloudsearch-domain.Search
DEBUG:botocore.hooks:Changing event name from docs.*.autoscaling.CreateLaunchConfiguration.complete-section to docs.*.auto-scaling.CreateLaunchConfiguration.complete-section
DEBUG:botocore.hooks:Changing event name from before-parameter-buil

In [14]:
endpoint_step = steps.EndpointStep(
    "Create Endpoint",
    endpoint_name=execution_input['EndpointName'],
    endpoint_config_name=execution_input['ModelName']
)

DEBUG:botocore.hooks:Changing event name from creating-client-class.iot-data to creating-client-class.iot-data-plane
DEBUG:botocore.hooks:Changing event name from before-call.apigateway to before-call.api-gateway
DEBUG:botocore.hooks:Changing event name from request-created.machinelearning.Predict to request-created.machine-learning.Predict
DEBUG:botocore.hooks:Changing event name from before-parameter-build.autoscaling.CreateLaunchConfiguration to before-parameter-build.auto-scaling.CreateLaunchConfiguration
DEBUG:botocore.hooks:Changing event name from before-parameter-build.route53 to before-parameter-build.route-53
DEBUG:botocore.hooks:Changing event name from request-created.cloudsearchdomain.Search to request-created.cloudsearch-domain.Search
DEBUG:botocore.hooks:Changing event name from docs.*.autoscaling.CreateLaunchConfiguration.complete-section to docs.*.auto-scaling.CreateLaunchConfiguration.complete-section
DEBUG:botocore.hooks:Changing event name from before-parameter-buil

In [15]:
workflow_definition = steps.Chain([
    training_step,
    model_step,
    endpoint_config_step,
    endpoint_step
])

In [16]:


workflow = Workflow(
    name='{}-{}'.format(model_name, timestamp),
    definition=workflow_definition,
    role=role,
    execution_input=execution_input
)

DEBUG:botocore.hooks:Event choose-service-name: calling handler <function handle_service_name_alias at 0x105344c10>
DEBUG:botocore.loaders:Loading JSON file: /Users/lanre.bakare/Documents/internal-projects/terraform-aws-mlops-module/venv/lib/python3.10/site-packages/botocore/data/stepfunctions/2016-11-23/service-2.json
DEBUG:botocore.loaders:Loading JSON file: /Users/lanre.bakare/Documents/internal-projects/terraform-aws-mlops-module/venv/lib/python3.10/site-packages/botocore/data/stepfunctions/2016-11-23/endpoint-rule-set-1.json.gz
DEBUG:botocore.hooks:Event creating-client-class.sfn: calling handler <function add_generate_presigned_url at 0x105293a30>
DEBUG:botocore.configprovider:Looking for endpoint for stepfunctions via: environment_service
DEBUG:botocore.configprovider:Looking for endpoint for stepfunctions via: environment_global
DEBUG:botocore.configprovider:Looking for endpoint for stepfunctions via: config_service
DEBUG:botocore.configprovider:Looking for endpoint for stepfun

In [17]:
workflow.render_graph()

In [18]:
workflow.create()

DEBUG:botocore.hooks:Event before-parameter-build.sfn.CreateStateMachine: calling handler <function generate_idempotent_uuid at 0x1053460e0>
DEBUG:botocore.regions:Calling endpoint provider with parameters: {'Region': 'us-east-1', 'UseDualStack': False, 'UseFIPS': False}
DEBUG:botocore.regions:Endpoint provider result: https://states.us-east-1.amazonaws.com
DEBUG:botocore.hooks:Event before-call.sfn.CreateStateMachine: calling handler <function add_recursion_detection_header at 0x105345d80>
DEBUG:botocore.hooks:Event before-call.sfn.CreateStateMachine: calling handler <function inject_api_version_header_if_needed at 0x105347910>
DEBUG:botocore.endpoint:Making request for OperationModel(name=CreateStateMachine) with params: {'url_path': '/', 'query_string': '', 'method': 'POST', 'headers': {'X-Amz-Target': 'AWSStepFunctions.CreateStateMachine', 'Content-Type': 'application/x-amz-json-1.0', 'User-Agent': 'Boto3/1.28.57 md/Botocore#1.31.57 ua/2.0 os/macos#23.6.0 md/arch#arm64 lang/python#

ClientError: An error occurred (AccessDeniedException) when calling the CreateStateMachine operation: Neither the global service principal states.amazonaws.com, nor the regional one is authorized to assume the provided role.

In [ ]:
execution = workflow.execute(
    inputs={
        #TODO those names should come from Terraform
        'JobName': 'regression-{}'.format(uuid.uuid1().hex), # Each Sagemaker Job requires a unique name
        'ModelName': 'regression-{}'.format(uuid.uuid1().hex), # Each Model requires a unique name,
        'EndpointName': 'regression-{}'.format(uuid.uuid1().hex) # Each Endpoint requires a unique name,
    }
)

In [ ]:
execution.render_progress()